## 1. Configuração Inicial

Primeiro, vamos instalar e importar as bibliotecas necessárias.

In [ ]:
# Instalação das bibliotecas (execute apenas uma vez)
!pip install openai numpy scipy matplotlib scikit-learn python-dotenv -q

In [ ]:
import warnings
from dotenv import load_dotenv
from openai import OpenAI

# Importando funções do módulo local
from embedding_utils import (
    get_embedding,
    cosine_similarity,
    plot_similarity_matrix,
    plot_embeddings_2d,
    comparar_palavras,
)

warnings.filterwarnings("ignore")

# Carrega variáveis do arquivo .env
load_dotenv()

print("✅ Bibliotecas importadas com sucesso!")

In [ ]:
# A chave da API é carregada automaticamente do arquivo .env
client = OpenAI()

print("✅ Cliente OpenAI configurado!")

## 2. Grupos de Palavras Relacionadas

Vamos definir três grupos de palavras semanticamente relacionadas:
- 🍎 **Frutas**: palavras relacionadas a frutas
- 🚗 **Veículos**: palavras relacionadas a transporte
- 🐶 **Animais**: palavras relacionadas a animais

In [ ]:
# Definindo grupos de palavras relacionadas
grupos = {
    "🍎 Frutas": ["maçã", "banana", "laranja", "morango", "uva"],
    "🚗 Veículos": ["carro", "moto", "bicicleta", "ônibus", "caminhão"],
    "🐶 Animais": [
        "cão",
        "gato",
        "vaca",
        "cavalo",
        "ovelha",
    ],
}

# Vamos visualizar os grupos
for grupo, palavras in grupos.items():
    print(f"{grupo}: {', '.join(palavras)}")

## 3. Gerando Embeddings para Cada Palavra

Agora vamos gerar os embeddings para todas as palavras.

In [ ]:
# Gerando embeddings para todas as palavras
embeddings = {}
todas_palavras = []
todos_embeddings = []
cores = []

cor_por_grupo = {"🍎 Frutas": "red", "🚗 Veículos": "blue", "🐶 Animais": "green"}

print("Gerando embeddings...\n")

for grupo, palavras in grupos.items():
    print(f"Processando {grupo}...")
    for palavra in palavras:
        embedding = get_embedding(client, palavra)
        embeddings[palavra] = {"embedding": embedding, "grupo": grupo}
        todas_palavras.append(palavra)
        todos_embeddings.append(embedding)
        cores.append(cor_por_grupo[grupo])
        print(f"  ✓ {palavra} (vetor com {len(embedding)} dimensões)")

print(f"\n✅ Total: {len(embeddings)} embeddings gerados!")
print(f"📐 Cada embedding tem {len(embedding)} dimensões")

## 4. Calculando Similaridade Entre Palavras

Vamos calcular a similaridade de cosseno entre algumas palavras para demonstrar que:
- Palavras do **mesmo grupo** têm alta similaridade
- Palavras de **grupos diferentes** têm baixa similaridade

In [ ]:
# Comparando palavras do MESMO grupo
print("═" * 50)
print("🔵 SIMILARIDADE ENTRE PALAVRAS DO MESMO GRUPO")
print("═" * 50)

comparacoes_mesmo_grupo = [
    ("uva", "morango"),
    ("carro", "moto"),
    ("cão", "gato"),
]

for palavra1, palavra2 in comparacoes_mesmo_grupo:
    sim = cosine_similarity(
        embeddings[palavra1]["embedding"], embeddings[palavra2]["embedding"]
    )
    print(f"{palavra1} ↔ {palavra2}: {sim:.4f} ({sim*100:.1f}%)")

In [ ]:
# Comparando palavras de GRUPOS DIFERENTES
print("═" * 50)
print("🔴 SIMILARIDADE ENTRE PALAVRAS DE GRUPOS DIFERENTES")
print("═" * 50)

comparacoes_grupos_diferentes = [
    ("uva", "caminhão"),
    ("banana", "cão"),
    ("ônibus", "vaca"),
]

for palavra1, palavra2 in comparacoes_grupos_diferentes:
    sim = cosine_similarity(
        embeddings[palavra1]["embedding"], embeddings[palavra2]["embedding"]
    )
    print(f"{palavra1} ↔ {palavra2}: {sim:.4f} ({sim*100:.1f}%)")

## 5. Matriz de Similaridade

Vamos criar uma matriz visual mostrando a similaridade entre todas as palavras.

In [ ]:
# Chamando a função plot_similarity_matrix (importada de embedding_utils.py)
matriz = plot_similarity_matrix(todas_palavras, embeddings, group_separators=[4.5, 9.5])

print("\n📊 Observe como os blocos na diagonal (mesmo grupo) são mais verdes!")

## 6. Visualização 2D com t-SNE

Os embeddings têm muitas dimensões (1536). Vamos usar **t-SNE** para reduzir para 2 dimensões e visualizar.

In [ ]:
# Configuração da legenda
legenda_grupos = [
    {"color": "red", "label": "Frutas"},
    {"color": "blue", "label": "Veículos"},
    {"color": "green", "label": "Animais"},
]

# Chamando a função plot_embeddings_2d (importada de embedding_utils.py)
embeddings_2d = plot_embeddings_2d(
    todas_palavras, todos_embeddings, cores, legenda_config=legenda_grupos
)

print("\n🎯 Note como palavras do mesmo grupo se agrupam!")

## 7. Experimento Interativo

Vamos testar a comparação de palavras livremente!

In [ ]:
# Teste você mesmo! Experimente diferentes pares de palavras:

comparar_palavras(client, "rei", "rainha")
comparar_palavras(client, "rei", "cadeira")
comparar_palavras(client, "cachorro", "gato")

## 📝 Resumo

Neste notebook, aprendemos:

1. **O que são embeddings**: representações vetoriais de texto
2. **Como gerar embeddings**: usando a API da OpenAI
3. **Similaridade de cosseno**: mede o quão "próximos" dois vetores estão
4. **Propriedade semântica**: palavras relacionadas têm embeddings similares

### 🚀 Próximos passos

No próximo notebook (`02_embeddings_frases.ipynb`), vamos:
- Trabalhar com **frases e parágrafos** (não apenas palavras)
- Criar um sistema de **busca semântica**
- Entender a base do **RAG (Retrieval Augmented Generation)**!